# Importing libraries

In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

# Changing the way pandas displays floats

In [2]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Importing dataset

In [116]:
suicides = pd.read_csv('/Users/andressalomferrer/Desktop/ironhack/Projects/Project_3/Project-Week-3-Data-Thieves/your-project/suicides_cleaned')

In [117]:
suicides.head()

,Unnamed: 0,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
0,36,Albania,1992,male,35-54 years,12,343800,3.49000,709452584,251,Boomers
1,37,Albania,1992,male,15-24 years,9,263700,3.41000,709452584,251,Generation X
2,38,Albania,1992,male,55-74 years,5,159500,3.13000,709452584,251,Silent
3,39,Albania,1992,male,25-34 years,7,245500,2.85000,709452584,251,Boomers
4,40,Albania,1992,female,15-24 years,7,292400,2.39000,709452584,251,Generation X


# Checking dtypes

In [118]:
suicides.dtypes

Unnamed: 0            int64
Country              object
Year                  int64
Gender               object
Age                  object
Num_Suicides          int64
Population            int64
Suicides/100kPop    float64
GDP/Year              int64
GDP/Capita            int64
Generation           object
dtype: object

# Renaming columns 

In [119]:
suicides.rename(columns={" gdp_for_year ($) ": "GDP/Year", "gdp_per_capita ($)": "GDP/Capita", "country": "Country",
                        "year": "Year", "sex": "Gender", "suicides_no": "Num_Suicides", "age": "Age", "population":
                        "Population", "suicides/100k pop": "Suicides/100kPop", "generation": "Generation", "HDI for year":
                         "HDI/Year"}, inplace=True)

In [120]:
suicides.head()

,Unnamed: 0,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
0,36,Albania,1992,male,35-54 years,12,343800,3.49000,709452584,251,Boomers
1,37,Albania,1992,male,15-24 years,9,263700,3.41000,709452584,251,Generation X
2,38,Albania,1992,male,55-74 years,5,159500,3.13000,709452584,251,Silent
3,39,Albania,1992,male,25-34 years,7,245500,2.85000,709452584,251,Boomers
4,40,Albania,1992,female,15-24 years,7,292400,2.39000,709452584,251,Generation X


In [123]:
suicides = suicides.astype({"GDP/Year": 'float'})

In [124]:
suicides.dtypes

Unnamed: 0            int64
Country              object
Year                  int64
Gender               object
Age                  object
Num_Suicides          int64
Population            int64
Suicides/100kPop    float64
GDP/Year            float64
GDP/Capita            int64
Generation           object
dtype: object

In [125]:
suicides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24612 entries, 0 to 24611
Data columns (total 11 columns):
Unnamed: 0          24612 non-null int64
Country             24612 non-null object
Year                24612 non-null int64
Gender              24612 non-null object
Age                 24612 non-null object
Num_Suicides        24612 non-null int64
Population          24612 non-null int64
Suicides/100kPop    24612 non-null float64
GDP/Year            24612 non-null float64
GDP/Capita          24612 non-null int64
Generation          24612 non-null object
dtypes: float64(2), int64(5), object(4)
memory usage: 2.1+ MB


In [127]:
# Checking null values 
suicides.isnull().sum()

Unnamed: 0          0
Country             0
Year                0
Gender              0
Age                 0
Num_Suicides        0
Population          0
Suicides/100kPop    0
GDP/Year            0
GDP/Capita          0
Generation          0
dtype: int64

In [128]:
suicides.describe()

,Unnamed: 0,Year,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita
count,24612.00000,24612.00000,24612.00000,24612.00000,24612.00000,24612.00000,24612.00000
mean,13929.65017,2002.91809,247.35787,1859937.87762,12.91848,471326282308.23108,17866.15651
std,8014.51989,7.19399,927.35144,3945249.44288,19.13124,1519293442925.49414,19568.29462
min,36.00000,1990.00000,0.00000,278.00000,0.00000,46919625.00000,251.00000
25%,7034.75000,1997.00000,3.00000,100754.75000,0.96000,10109612142.00000,3724.00000
50%,13933.50000,2003.00000,26.00000,444998.50000,5.98000,51264390116.00000,10220.00000
75%,20880.25000,2009.00000,132.00000,1497711.50000,16.66000,273317737047.00000,26671.00000
max,27819.00000,2015.00000,22338.00000,43805214.00000,224.97000,18120714000000.00000,126352.00000


In [129]:
suicides[suicides['Num_Suicides'] == 0][0:3]

,Unnamed: 0,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
8,44,Albania,1992,female,5-14 years,0,336700,0.00000,709452584.00000,251,Millenials
9,45,Albania,1992,female,75+ years,0,38700,0.00000,709452584.00000,251,G.I. Generation
10,46,Albania,1992,male,5-14 years,0,362900,0.00000,709452584.00000,251,Millenials


In [130]:
driver = 'mysql+pymysql'
host = "34.90.32.189"
username = 'root'
password = '123456789'
db = "suicides1"
connection_string = f'{driver}://{username}:{password}@{host}/{db}'
connection=create_engine(connection_string)
engine = create_engine(connection_string)

In [131]:
suicides.to_sql("suicides1", con=connection, if_exists='replace')

In [132]:
suicides_year = suicides.groupby("Year").sum().sort_values("Num_Suicides", ascending=False)

In [133]:
suicides_year = suicides_year.reset_index()

In [134]:
suicides_year.sort_values('Year', ascending = False)

,Year,Unnamed: 0,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita
23,2015,11053140,203640,1774657932,8253.99000,580433884300596.00000,19516008
20,2014,13684836,222984,1912057309,10306.73000,672496725417372.00000,25665252
19,2013,13570920,223199,1890161710,10663.64000,685050211812900.00000,26911368
18,2012,13939746,230160,1912812088,11101.91000,676502580180480.00000,26058300
12,2011,14368668,236484,1993362332,11367.84000,678000047330112.00000,26936148
11,2010,15288936,238702,1997297329,11843.99000,625862405486292.00000,25193196
8,2009,15189042,243487,1976228366,12176.04000,584833269596760.00000,24145248
13,2008,14231178,235447,1860620851,12145.84000,619322403842760.00000,26936208
15,2007,14104116,233408,1859564353,12410.15000,576509655876756.00000,24709620
16,2006,14306106,233361,1840908837,12166.01000,517865365084944.00000,21563784


In [136]:
suicides.dtypes

Unnamed: 0            int64
Country              object
Year                  int64
Gender               object
Age                  object
Num_Suicides          int64
Population            int64
Suicides/100kPop    float64
GDP/Year            float64
GDP/Capita            int64
Generation           object
dtype: object

# NO ME CUADRA:

In [24]:
## es lo que aparece en la tabla de arriba, pero no es muy poco? 

pop15 = suicides[suicides['Year'] == 2015]['Population'].sum()

In [25]:
pop15

1774657932

# ------

In [26]:
# Q4) Males or females commit suicide more in 2015?

# QUESTION 4:

# Males or females commit suicide more in 2015?

In [27]:
# To answer this question we are going to create a dataframe for the year 2015:

suicides_2015 = suicides[suicides['Year'] == 2015]

In [28]:
# Now we are going to filter by the columns we need:

s_gender2015 = suicides_2015[['Gender','Num_Suicides']].groupby('Gender').sum()

## As we can see, the number of suicides in 2015 is higher in males than in females:


In [29]:
s_gender2015

,Num_Suicides
Gender,
female,47248
male,156392


In [30]:
# Top 10 countries with more suicides in year 2015 taking into account the number of suicides.

In [31]:
top10countriesbynum = suicides_2015[['Country','Num_Suicides']].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [32]:
top10countriesbynum

,Num_Suicides
Country,
United States,44189
Russian Federation,25432
Japan,23092
Republic of Korea,13510
Brazil,11163
Germany,10088
Ukraine,7574
Mexico,6234
Poland,5420


# Now we realize that took into account the num of suicides. But if we take into account the size of the population, results can be different. Let's check:




In [33]:
top10bysize = suicides_2015[['Country','Suicides/100kPop']].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]

In [34]:
top10bysize

,Suicides/100kPop
Country,
Republic of Korea,389.14000
Lithuania,385.48000
Uruguay,270.02000
Slovenia,264.81000
Hungary,257.27000
Latvia,251.47000
Ukraine,244.72000
Kazakhstan,238.02000
Russian Federation,236.92000


### Taking the size of the population into account, we realize that only  Russia, Ukraine and Republic of Korea appears in this new ranking.

In [35]:
# Creating an intersection by index to see which countries appears in both rankings.
idx1 = pd.Index(top10bysize.index)
idx2 = pd.Index(top10countriesbynum.index)

In [36]:
set_intersection = idx1.intersection(idx2)

In [37]:
set_intersection

Index(['Republic of Korea', 'Ukraine', 'Russian Federation'], dtype='object', name='Country')

## Now we will do the same but by gender:

### MALES : TOP 10 COUNTRIES BY NUM

In [38]:
top10malesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='male'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [39]:
top10malesbynum

,Num_Suicides
Country,
United States,33990
Russian Federation,20861
Japan,16146
Republic of Korea,9556
Brazil,8766
Germany,7403
Ukraine,6148
Mexico,4986
Poland,4697


### FEMALES : TOP 10 COUNTRIES BY NUM

In [40]:
top10femalesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='female'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [41]:
top10femalesbynum

,Num_Suicides
Country,
United States,10199
Japan,6946
Russian Federation,4571
Republic of Korea,3954
Germany,2685
Brazil,2397
Ukraine,1426
Mexico,1248
United Kingdom,1178


In [42]:
# Creating an intersection by index to see if the countries with a higher number of suicides
# are the same countries for both genres.

idgnf = pd.Index(top10femalesbynum.index)
idgnm = pd.Index(top10malesbynum.index)

In [43]:
set_intersection = idgnf.intersection(idgnm)

In [44]:
set_intersection

Index(['United States', 'Japan', 'Russian Federation', 'Republic of Korea',
       'Germany', 'Brazil', 'Ukraine', 'Mexico', 'United Kingdom'],
      dtype='object', name='Country')

### As we can see 9 out of 10 countries match

In [45]:
# Spain appears in top10 of countries where females commit suicides.

idgnf.difference(idgnm)

Index(['Spain'], dtype='object', name='Country')

In [46]:
# Poland appears in top10 of countries where males commit suicides.

idgnm.difference(idgnf)

Index(['Poland'], dtype='object', name='Country')

### MALE: TOP 10 COUNTRIES BY SIZE


In [47]:
top10malesbysize = suicides_2015[['Country','Gender','Suicides/100kPop']].loc[suicides_2015['Gender']=='male'].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]


In [48]:
top10malesbysize

,Suicides/100kPop
Country,
Lithuania,323.31000
Republic of Korea,285.19000
Uruguay,223.03000
Slovenia,217.67000
Latvia,211.07000
Ukraine,206.64000
Hungary,203.39000
Russian Federation,199.26000
Kazakhstan,195.35000


### FEMALE: TOP 10 COUNTRIES BY SIZE

In [49]:
top10femalesbysize = suicides_2015[['Country','Gender','Suicides/100kPop']].loc[suicides_2015['Gender']=='female'].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]


In [50]:
top10femalesbysize

,Suicides/100kPop
Country,
Republic of Korea,103.95000
Lithuania,62.17000
Japan,58.29000
Hungary,53.88000
Belgium,52.21000
Croatia,51.63000
Slovenia,47.14000
Uruguay,46.99000
Iceland,45.69000


In [51]:
# Creating an intersection by index to see if the countries with a higher number of suicides 
# by size of the population are the same countries for both genres.

idgsf = pd.Index(top10femalesbysize.index)
idgsm = pd.Index(top10malesbysize.index)

In [52]:
set_intersection = idgsf.intersection(idgsm)

In [53]:
set_intersection

Index(['Republic of Korea', 'Lithuania', 'Hungary', 'Croatia', 'Slovenia',
       'Uruguay'],
      dtype='object', name='Country')

In [54]:
# females but no males:

idgsf.difference(idgsm)

Index(['Belgium', 'Iceland', 'Japan', 'Serbia'], dtype='object', name='Country')

In [55]:
# males but not females:

idgsm.difference(idgsf)

Index(['Kazakhstan', 'Latvia', 'Russian Federation', 'Ukraine'], dtype='object', name='Country')

### In this case, the countries Republic of Korea,  Lithuania, Hungary, Croatia, Slovenia and Uruguay match for both genres.

### But Belgiun, Iceland, Japan and Serbia appear in the top 10 for females while Kazakhstan, Latvia, Russian Federation and Ukraine appear in the top 10 for males.

# QUESTION 5: 

#  Which generation has committed suicide the most/least?


In [56]:
# checking the data 

suicides_2015.head(2)

,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
576,Antigua and Barbuda,2015,female,55-74 years,1,6403,15.62000,1364863037.00000,14853,Boomers
577,Antigua and Barbuda,2015,female,15-24 years,0,8561,0.00000,1364863037.00000,14853,Millenials


## Let's check the number of suicides grouped by generations:

## First, we will see which generation committed more suicides from 2000 till 2015

In [73]:
# Change 'Year' to integer so we can filter by years.
suicides_new = suicides.astype({"Year": 'int64'})

In [113]:
# This will be our filter
mask = (suicides_new['Year'] >= 1990) & (suicides_new['Year'] < 2016)

In [114]:
df = suicides_new.loc[mask]

In [115]:
# new data frame from 1990 till 2015 
df.head(2)

,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
36,Albania,1992,male,35-54 years,12,343800,3.49000,709452584.00000,251,Boomers
37,Albania,1992,male,15-24 years,9,263700,3.41000,709452584.00000,251,Generation X


In [100]:
df1 = df[['Generation', 'Age','Num_Suicides']]

In [101]:
df1.groupby(['Generation','Age']).sum()

Num_Suicides
Generation      Age                      
Boomers         35-54 years        923044
                55-74 years        286601
G.I. Generation 75+ years           21633
Generation X    15-24 years         31742
                25-34 years        430755
                35-54 years        485848
Generation Z    5-14 years          15906
Millenials      15-24 years        413035
                25-34 years        177128
                5-14 years          13465
Silent          55-74 years        645749
                75+ years          349860

In [102]:
# we will drop the column 'Age' because can be confuse as is taking two different ranges
# fot the same generation. The reason why it does this is because in 2000 people from the same
# generation was in another range of age.

In [103]:
df1 = df1.groupby('Generation').sum().sort_values("Num_Suicides", ascending=False)

In [104]:
df1

,Num_Suicides
Generation,
Boomers,1209645
Silent,995609
Generation X,948345
Millenials,603628
G.I. Generation,21633
Generation Z,15906


### ----- COUNT CUANTOS DE CADA GENERATION HAY ------

In [106]:
df2 = df.groupby('Generation').sum().sort_values("Num_Suicides", ascending=False)

In [109]:
df2 = df2[['Population', 'Suicides/100kPop']]

In [110]:
df2

,Population,Suicides/100kPop
Generation,,
Boomers,7112476453,36854.62000
Silent,5076891426,84449.40000
Generation X,6908760170,36966.83000
Millenials,8025200090,30024.97000
G.I. Generation,86361616,4461.89000
Generation Z,2503541842,944.18000


In [111]:
df2['Population'].sum()

29713231597

In [66]:
# Let's see what happened in 2015:

In [67]:
sui_gen = suicides_2015[['Generation', 'Age','Num_Suicides']]

In [68]:
sui_gen.groupby(['Generation','Age']).sum()

Num_Suicides
Generation   Age                      
Boomers      55-74 years         53588
Generation X 35-54 years         71298
Generation Z 5-14 years           1681
Millenials   15-24 years         21841
             25-34 years         32118
Silent       75+ years           23114

## - - FALTA MERGE MILLENIALS ROWS - -

In [69]:
#Q6) Countries with the highest 5-14 years range suicide rate?

In [112]:
suicides.head()

,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
0,Albania,1987,male,15-24 years,21,312900,6.71000,2156624900.00000,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19000,2156624900.00000,796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83000,2156624900.00000,796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59000,2156624900.00000,796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28000,2156624900.00000,796,Boomers


In [ ]:
suicides_lrange = suicides